In [11]:
import sys
from genotypes import *
from graphviz import Digraph
import argparse

In [12]:
def parse_args():
    parser = argparse.ArgumentParser(description='Modality optimization.')
    parser.add_argument('--multiplier', type=int, help='cell output concat', default=4)
    parser.add_argument('--steps', type=int, help='cell steps', default=4)
    parser.add_argument('--num_input_nodes', type=int, help='cell input', default=8)
    parser.add_argument('--num_keep_edges', type=int, help='cell step connect', default=2)
    parser.add_argument('--node_multiplier', type=int, help='inner node output concat', default=1)
    parser.add_argument('--node_steps', type=int, help='inner node steps', default=2)
    return parser.parse_args("")

In [13]:
args = parse_args()

In [14]:
args

Namespace(multiplier=4, steps=4, num_input_nodes=8, num_keep_edges=2, node_multiplier=1, node_steps=2)

In [15]:
new_op_dict = {
    'sum': 'Sum', 
    'scale_dot_attn': 'ScaleDotAttn',
    'cat_conv_glu': 'LinearGLU',
    'cat_conv_relu': 'ConcatFC'
}

In [16]:
def change_to_new_op(inner_steps):
    ret = []
    for op in inner_steps:
        ret.append(new_op_dict[op])
    return ret

In [17]:
def change_to_new_genotype(genotype):
    new_edges = genotype.edges
    new_steps = []
    new_concat = genotype.concat
    for step in genotype.steps:
        new_inner_edges = step.inner_edges    
        new_inner_steps = change_to_new_op(step.inner_steps)
        new_inner_concat = step.inner_concat
        new_step = StepGenotype(inner_edges = new_inner_edges,
                                inner_steps = new_inner_steps,
                                inner_concat = new_inner_concat)
        new_steps.append(new_step)
    new_genotype = Genotype(edges = new_edges, steps = new_steps, concat = new_concat)
    return new_genotype

In [8]:
def plot(genotype, filename, args, task=None):
    
    if genotype == None:
        return 

    multiplier = args.multiplier
    num_input_nodes = args.num_input_nodes
    num_keep_edges = args.num_keep_edges
    
    node_steps = args.node_steps
    node_multiplier = args.node_multiplier
    
    g = Digraph(
            format='pdf',
            edge_attr=dict(fontsize='20', fontname="times", penwidth='1.5'),
            node_attr=dict(style='rounded, filled', shape='rect', align='center', 
                           fontsize='20', height='0.5', width='0.5', penwidth='2', 
                           fontname="helvetica"),
            engine='dot')
#     g.attr(style='rounded, filled', color='red')
#     g.body.extend(['rankdir=LR'])
    g.attr(rankdir='LR')

    input_nodes = []
    input_nodes_A = []
    input_nodes_B = []
    
    nc = Digraph(node_attr={'shape': 'box'})
    nc.attr(rankdir='BT')
    nc.attr(rank='same')
#     nc.attr(style='rounded, filled', color='red')
#                    fontsize='20', align='center')
#     with nc.subgraph(name='cluster_input_features', node_attr={'shape': 'box'}) as c:
#         c.attr(style='rounded, filled', color='red', 
#                    fontsize='20', align='center')
#         c.attr(rankdir='BT')
#         c.attr(rank='same')
#         c.attr(constraint='false')
    assert len(genotype.edges) % num_keep_edges == 0
    steps = len(genotype.edges) // num_keep_edges
    
    with g.subgraph() as nothing:
        c = nc
        
        input_nodes_A = []
        input_nodes_B = []
        
        with c.subgraph(name='cluster_video_features', node_attr={'shape': 'box'}) as ca:
            ca.attr(style='rounded, filled', color='lightgrey', 
                           fontsize='20', align='center')

            input_nodes_A = ["Video_1", "Video_2", "Video_3", "Video_4"]
            if task == 'mmimdb':
                input_nodes_A = ["Image_1", "Image_2", "Image_3", "Image_4"]
            
            if task == 'nvgesture' or task == 'ego':
                input_nodes_A = ["RGB_1", "RGB_2", "RGB_3", "RGB_4"]
                
            for input_node in input_nodes_A:
                ca.node(input_node, fillcolor='lightskyblue1')
            
            for i in range(len(input_nodes_A)-1):
                c.edge(input_nodes_A[i], input_nodes_A[i+1], label=None)
            
            
    
        with c.subgraph(name='cluster_skeleton_features', node_attr={'shape': 'box'}) as cb:
            cb.attr(style='rounded, filled', color='lightgrey', 
                           fontsize='20', align='center')

            input_nodes_B = ["Skeleton_1", "Skeleton_2", "Skeleton_3", "Skeleton_4"]
            if task == 'mmimdb':
                input_nodes_B = ["Text_1", "Text_2"]
            
            if task == 'nvgesture' or task == 'ego':
                input_nodes_B = ["Depth_1", "Depth_2", "Depth_3", "Depth_4"]

            for input_node in input_nodes_B:
                cb.node(input_node, fillcolor='darkolivegreen1')
            
            for i in range(len(input_nodes_B)-1):
                c.edge(input_nodes_B[i], input_nodes_B[i+1], label=None)
                
#                 for j in range(steps):        
#                     node_x_name = "X_C{}".format(j)
#                     node_y_name = "Y_C{}".format(j)
# #                     g.edge(input_nodes_B[i], node_y_name, style='invis')
#                     g.edge(input_nodes_B[i], node_x_name, style='invis')
            
        c.edge(input_nodes_B[-1], input_nodes_A[0], style='invis')
        
    g.subgraph(nc)
    
    input_nodes = input_nodes_A + input_nodes_B
    assert len(input_nodes) == num_input_nodes

    node_names = [] 
    node_names += input_nodes

    for i in range(steps):
        # step_op = genotype.steps[i][0]
        # step_node_name = "{}_{}".format(i, step_op)
        node_z_name = "Z_C{}".format(i+1)
        node_names.append(node_z_name)
    
    for i in range(steps-1):        
        node_x_name = "X_C{}".format(i+2)
        node_y_name = "Y_C{}".format(i+1)
#         g.edge(node_x_name, node_y_name, style='invis')
#         g.edge(node_y_name, node_x_name, style='invis')

    
#     for i in genotype.concat[0:-1]:
# #         print(i)
#         g.edge(node_names[i+1], node_names[i], style='invis')
        
    for i in range(steps):
        # step_op = genotype.steps[i][0]
        # step_node_name = "{}_{}".format(i, step_op)
        step_node_name = "cluster_step_{}".format(i)
        step_gene = genotype.steps[i]
        
        node_x_name = "X_C{}".format(i+1)
        node_y_name = "Y_C{}".format(i+1)
        node_z_name = "Z_C{}".format(i+1)

        with g.subgraph(name=step_node_name, node_attr={'shape': 'box'}) as c:
            c.attr(style='rounded, filled', color='tan1', 
                   fontsize='20', align='center')
            c.node_attr.update(style='rounded, filled')
            
            inner_node_names = [node_x_name, node_y_name]
            for j in range(node_steps):    
#                 print(i, j)
#                 print(step_gene)
                inner_step_name = "C{}_S{}\n{}".format(i+1, j+1, step_gene.inner_steps[j]) 
                inner_node_names.append(inner_step_name)
            
            with c.subgraph() as ic:
#                 ic.attr(rankdir='BT')
#                 ic.attr(rank='same')
                for inner_node_name in inner_node_names:
                    if inner_node_name != node_x_name and inner_node_name != node_y_name:
                        ic.node(inner_node_name, fillcolor='khaki1')
            
            c.node(node_x_name, fillcolor='maroon2')
            c.node(node_y_name, fillcolor='green3')
            c.node(node_z_name, fillcolor='purple')
            
#             c.edge(input_nodes_B[-1], node_x_name, style='invis')
#             c.edge(input_nodes_B[-1], node_y_name, style='invis')
#             for in_A in input_nodes_A:
#                 c.edge(in_A, node_x_name, style='invis')
#                 c.edge(in_A, node_y_name, style='invis')
            
            for in_B in input_nodes_B:
                c.edge(in_B, node_x_name, style='invis')
                c.edge(in_B, node_y_name, style='invis')

            # print(inner_node_names)
            for j in range(node_steps):
                x = step_gene.inner_edges[2*j][1]
                x_op = step_gene.inner_edges[2*j][0]
                y = step_gene.inner_edges[2*j+1][1]
                y_op = step_gene.inner_edges[2*j+1][0]
                # print(j, x, x_op, y, y_op)
                # c.edge(inner_node_names[x], inner_node_names[2+j], label=x_op)
                # c.edge(inner_node_names[y], inner_node_names[2+j], label=y_op)
                c.edge(inner_node_names[x], inner_node_names[2+j], label=None)
                c.edge(inner_node_names[y], inner_node_names[2+j], label=None)
                
#                 c.edge(inner_node_names[2+j], inner_node_names[x], label=None)
#                 c.edge(inner_node_names[2+j], inner_node_names[y], label=None)
            
            for j in range(args.node_multiplier):
                # c.edge(inner_node_names[-(j+1)], node_z_name, label='skip')
                c.edge(inner_node_names[-(j+1)], node_z_name, label=None)
            # skip connection
#             c.edge(node_x_name, node_z_name, label=None)
        
        edge_x_op = genotype.edges[2*i][0]
        edge_x_from = node_names[genotype.edges[2*i][1]]
        edge_x_to = node_x_name
        # g.edge(edge_x_from, edge_x_to, label=edge_x_op)
        g.edge(edge_x_from, edge_x_to, label=None, color="blue")

        edge_y_op = genotype.edges[2*i+1][0]
        edge_y_from = node_names[genotype.edges[2*i+1][1]]
        edge_y_to = node_y_name
        # g.edge(edge_y_from, edge_y_to, label=edge_y_op)
        g.edge(edge_y_from, edge_y_to, label=None, color="blue")
    
    g.node("Reduction\nOutput", fillcolor='grey91')
    

    for i in genotype.concat:
        g.edge(node_names[i], "Reduction\nOutput", color="blue")

    g.render(filename, view=True)

In [9]:
# # best mmimd model
# args.steps=2
# args.multiplier=2
# args.num_input_nodes=6
# args.num_keep_edges=2
# args.node_multiplier=2 
# args.node_steps=1

# task = 'mmimdb'
# genotype = Genotype(edges=[('skip', 2), ('skip', 4), ('skip', 4), ('skip', 6)], steps=[StepGenotype(inner_edges=[('skip', 1), ('skip', 0)], inner_steps=['cat_conv_relu'], inner_concat=[2]), StepGenotype(inner_edges=[('skip', 1), ('skip', 0)], inner_steps=['cat_conv_glu'], inner_concat=[2])], concat=[6, 7])
# new_genotype = change_to_new_genotype(genotype)
# model_name = 'best_mmimdb_model'
# plot(new_genotype, "/Users/v_yinyihang/Desktop/CVPR2021-MultiModalNas/sup_figures/" + model_name, args, task)

In [10]:
# # best ntu model
# args.steps=2
# args.multiplier=2
# args.num_input_nodes=8
# args.num_keep_edges=2
# args.node_multiplier=2 
# args.node_steps=2

# genotype = Genotype(edges=[('skip', 2), ('skip', 7), ('skip', 2), ('skip', 3)], steps=[StepGenotype(inner_edges=[('skip', 0), ('skip', 1), ('skip', 2), ('skip', 0)], inner_steps=['cat_conv_glu', 'cat_conv_glu'], inner_concat=[2, 3]), StepGenotype(inner_edges=[('skip', 0), ('skip', 1), ('skip', 2), ('skip', 0)], inner_steps=['scale_dot_attn', 'scale_dot_attn'], inner_concat=[2, 3])], concat=[8, 9])
# new_genotype = change_to_new_genotype(genotype)
# model_name = 'best_ntu_model'
# plot(new_genotype, "/Users/v_yinyihang/Desktop/CVPR2021-MultiModalNas/sup_figures/" + model_name, args, 'ntu')

In [110]:
# # best ego model
# args.steps=2
# args.multiplier=2
# args.num_input_nodes=8
# args.num_keep_edges=2
# args.node_multiplier=3 
# args.node_steps=3

# task = 'ego'
# genotype = Genotype(edges=[('skip', 1), ('skip', 5), ('skip', 4), ('skip', 5)], steps=[StepGenotype(inner_edges=[('skip', 0), ('skip', 1), ('skip', 1), ('skip', 0), ('skip', 3), ('skip', 2)], inner_steps=['sum', 'sum', 'sum'], inner_concat=[2, 3, 4]), StepGenotype(inner_edges=[('skip', 0), ('skip', 1), ('skip', 1), ('skip', 0), ('skip', 2), ('skip', 3)], inner_steps=['sum', 'sum', 'sum'], inner_concat=[2, 3, 4])], concat=[8, 9])
# new_genotype = change_to_new_genotype(genotype)
# model_name = 'best_ego_model'
# plot(new_genotype, "/Users/v_yinyihang/Desktop/CVPR2021-MultiModalNas/sup_figures/" + model_name, args, task)

In [111]:
# # best ntu model
# args.steps=2
# args.multiplier=2
# args.num_input_nodes=8
# args.num_keep_edges=2
# args.node_multiplier=2 
# args.node_steps=2

# genotype = Genotype(edges=[('skip', 2), ('skip', 7), ('skip', 2), ('skip', 3)], steps=[StepGenotype(inner_edges=[('skip', 0), ('skip', 1), ('skip', 2), ('skip', 0)], inner_steps=['cat_conv_glu', 'cat_conv_glu'], inner_concat=[2, 3]), StepGenotype(inner_edges=[('skip', 0), ('skip', 1), ('skip', 2), ('skip', 0)], inner_steps=['scale_dot_attn', 'scale_dot_attn'], inner_concat=[2, 3])], concat=[8, 9])
# new_genotype = change_to_new_genotype(genotype)
# model_name = 'ntu_config_1'
# plot(new_genotype, "/Users/v_yinyihang/Desktop/CVPR2021-MultiModalNas/sup_figures/" + model_name, args, 'ntu')

In [112]:
# # NTU TOP4 Settings
# # config 1 == best_ntu_model
# # config 2
# args.steps=2
# args.multiplier=2
# args.num_input_nodes=8
# args.num_keep_edges=2
# args.node_multiplier=1 
# args.node_steps=1

# genotype = Genotype(edges=[('skip', 2), ('skip', 3), ('skip', 2), ('skip', 7)], steps=[StepGenotype(inner_edges=[('skip', 1), ('skip', 0)], inner_steps=['cat_conv_relu'], inner_concat=[2]), StepGenotype(inner_edges=[('skip', 0), ('skip', 1)], inner_steps=['cat_conv_relu'], inner_concat=[2])], concat=[8, 9])
# new_genotype = change_to_new_genotype(genotype)
# model_name = 'ntu_config_2'
# plot(new_genotype, "/Users/v_yinyihang/Desktop/CVPR2021-MultiModalNas/sup_figures/" + model_name, args, 'ntu')

In [113]:
# args.steps=4
# args.multiplier=4
# args.num_input_nodes=8
# args.num_keep_edges=2
# args.node_multiplier=2
# args.node_steps=1

# genotype = Genotype(edges=[('skip', 2), ('skip', 7), ('skip', 3), ('skip', 7)], steps=[StepGenotype(inner_edges=[('skip', 1), ('skip', 0), ('skip', 2), ('skip', 1)], inner_steps=['cat_conv_glu', 'scale_dot_attn'], inner_concat=[2, 3]), StepGenotype(inner_edges=[('skip', 0), ('skip', 1), ('skip', 2), ('skip', 0)], inner_steps=['scale_dot_attn', 'scale_dot_attn'], inner_concat=[2, 3])], concat=[8, 9])
# new_genotype = change_to_new_genotype(genotype)
# model_name = 'ntu_config_3'
# plot(new_genotype, "/Users/v_yinyihang/Desktop/CVPR2021-MultiModalNas/sup_figures/" + model_name, args, 'ntu')

In [114]:
# args.steps=4
# args.multiplier=4
# args.num_input_nodes=8
# args.num_keep_edges=2
# args.node_multiplier=2
# args.node_steps=2

# genotype = Genotype(edges=[('skip', 2), ('skip', 3), ('skip', 2), ('skip', 7), ('skip', 2), ('skip', 5), ('skip', 2), ('skip', 8)], steps=[StepGenotype(inner_edges=[('skip', 0), ('skip', 1), ('skip', 1), ('skip', 0)], inner_steps=['sum', 'sum'], inner_concat=[3]), StepGenotype(inner_edges=[('skip', 1), ('skip', 0), ('skip', 1), ('skip', 0)], inner_steps=['cat_conv_relu', 'scale_dot_attn'], inner_concat=[3]), StepGenotype(inner_edges=[('skip', 0), ('skip', 1), ('skip', 0), ('skip', 1)], inner_steps=['cat_conv_relu', 'scale_dot_attn'], inner_concat=[3]), StepGenotype(inner_edges=[('skip', 1), ('skip', 0), ('skip', 0), ('skip', 1)], inner_steps=['sum', 'sum'], inner_concat=[3])], concat=[8, 9, 10, 11])
# new_genotype = change_to_new_genotype(genotype)
# model_name = 'ntu_config_4'
# plot(new_genotype, "/Users/v_yinyihang/Desktop/CVPR2021-MultiModalNas/sup_figures/" + model_name, args, 'ntu')

In [115]:
# # best ego model 
# # config 1
# args.steps=2
# args.multiplier=2
# args.num_input_nodes=8
# args.num_keep_edges=2
# args.node_multiplier=3 
# args.node_steps=3

# task = 'ego'
# genotype = Genotype(edges=[('skip', 1), ('skip', 5), ('skip', 4), ('skip', 5)], steps=[StepGenotype(inner_edges=[('skip', 0), ('skip', 1), ('skip', 1), ('skip', 0), ('skip', 3), ('skip', 2)], inner_steps=['sum', 'sum', 'sum'], inner_concat=[2, 3, 4]), StepGenotype(inner_edges=[('skip', 0), ('skip', 1), ('skip', 1), ('skip', 0), ('skip', 2), ('skip', 3)], inner_steps=['sum', 'sum', 'sum'], inner_concat=[2, 3, 4])], concat=[8, 9])
# new_genotype = change_to_new_genotype(genotype)
# model_name = 'ego_config_1'
# plot(new_genotype, "/Users/v_yinyihang/Desktop/CVPR2021-MultiModalNas/sup_figures/" + model_name, args, task)

In [116]:
# # best ego model 
# # config 2
# args.steps=2
# args.multiplier=2
# args.num_input_nodes=8
# args.num_keep_edges=2
# args.node_multiplier=1 
# args.node_steps=1

# task = 'ego'
# genotype = Genotype(edges=[('skip', 1), ('skip', 5), ('skip', 5), ('skip', 8)], steps=[StepGenotype(inner_edges=[('skip', 0), ('skip', 1)], inner_steps=['scale_dot_attn'], inner_concat=[2]), StepGenotype(inner_edges=[('skip', 1), ('skip', 0)], inner_steps=['scale_dot_attn'], inner_concat=[2])], concat=[8, 9])
# new_genotype = change_to_new_genotype(genotype)
# model_name = 'ego_config_2'
# plot(new_genotype, "/Users/v_yinyihang/Desktop/CVPR2021-MultiModalNas/sup_figures/" + model_name, args, task)

In [30]:
# # best ego model 
# # config 3
# args.steps=2
# args.multiplier=2
# args.num_input_nodes=8
# args.num_keep_edges=2
# args.node_multiplier=4
# args.node_steps=4

# task = 'ego'
# genotype = Genotype(edges=[('skip', 4), ('skip', 5), ('skip', 5), ('skip', 8)], steps=[StepGenotype(inner_edges=[('skip', 1), ('skip', 0), ('skip', 2), ('skip', 1), ('skip', 3), ('skip', 2), ('skip', 4), ('skip', 3)], inner_steps=['scale_dot_attn', 'scale_dot_attn', 'scale_dot_attn', 'cat_conv_glu'], inner_concat=[2, 3, 4, 5]), StepGenotype(inner_edges=[('skip', 1), ('skip', 0), ('skip', 2), ('skip', 0), ('skip', 2), ('skip', 0), ('skip', 3), ('skip', 2)], inner_steps=['scale_dot_attn', 'cat_conv_relu', 'cat_conv_relu', 'scale_dot_attn'], inner_concat=[2, 3, 4, 5])], concat=[8, 9])
# new_genotype = change_to_new_genotype(genotype)
# model_name = 'ego_config_3'
# plot(new_genotype, "/Users/v_yinyihang/Desktop/CVPR2021-MultiModalNas/sup_figures/" + model_name, args, task)

In [31]:
# # best ego model 
# # config 4
# args.steps=3
# args.multiplier=3
# args.num_input_nodes=8
# args.num_keep_edges=2
# args.node_multiplier=2
# args.node_steps=2

# task = 'ego'
# genotype = Genotype(edges=[('skip', 1), ('skip', 4), ('skip', 0), ('skip', 4), ('skip', 4), ('skip', 9)], steps=[StepGenotype(inner_edges=[('skip', 0), ('skip', 1), ('skip', 2), ('skip', 1)], inner_steps=['sum', 'sum'], inner_concat=[2, 3]), StepGenotype(inner_edges=[('skip', 0), ('skip', 1), ('skip', 2), ('skip', 0)], inner_steps=['cat_conv_relu', 'cat_conv_relu'], inner_concat=[2, 3]), StepGenotype(inner_edges=[('skip', 1), ('skip', 0), ('skip', 2), ('skip', 1)], inner_steps=['cat_conv_glu', 'cat_conv_relu'], inner_concat=[2, 3])], concat=[8, 9, 10])
# new_genotype = change_to_new_genotype(genotype)
# model_name = 'ego_config_4'
# plot(new_genotype, "/Users/v_yinyihang/Desktop/CVPR2021-MultiModalNas/sup_figures/" + model_name, args, task)

In [3]:
a = Genotype(edges=[('skip', 2), ('skip', 7), ('skip', 2), ('skip', 3)], steps=[StepGenotype(inner_edges=[('skip', 0), ('skip', 1), ('skip', 2), ('skip', 0)], inner_steps=['LinearGLU', 'LinearGLU'], inner_concat=[2, 3]), StepGenotype(inner_edges=[('skip', 0), ('skip', 1), ('skip', 2), ('skip', 0)], inner_steps=['ScaleDotAttn', 'ScaleDotAttn'], inner_concat=[2, 3])], concat=[8, 9])

In [5]:
import pickle

In [6]:
def save_pickle(obj, obj_path):
    obj_file = open(obj_path, "wb")
    pickle.dump(obj, obj_file)
    obj_file.close()

In [16]:
path = '/Users/v_yinyihang/Desktop/yihang/Baidu_MM/BM-NAS/final_exp/ntu/search-C128_L8_N2_S2_NS2_NM2_drpt0.2-20200924-204321/best/best_genotype.pkl'

In [17]:
a

Genotype(edges=[('skip', 2), ('skip', 7), ('skip', 2), ('skip', 3)], steps=[StepGenotype(inner_edges=[('skip', 0), ('skip', 1), ('skip', 2), ('skip', 0)], inner_steps=['LinearGLU', 'LinearGLU'], inner_concat=[2, 3]), StepGenotype(inner_edges=[('skip', 0), ('skip', 1), ('skip', 2), ('skip', 0)], inner_steps=['ScaleDotAttn', 'ScaleDotAttn'], inner_concat=[2, 3])], concat=[8, 9])

In [18]:
save_pickle(a, path)

In [19]:
import utils

In [24]:
path = '/Users/v_yinyihang/Desktop/yihang/Baidu_MM/BM-NAS/best_exp/ntu/search-node_mul2_skip_v4_2step_C128-20200924-204321/best/best_genotype.pkl'

In [26]:
import models

ModuleNotFoundError: No module named 'models'

In [27]:
utils.load_pickle(path)

ModuleNotFoundError: No module named 'models'